# K-Means Lab


## Import required packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
report_df = pd.read_csv('WH Report_preprocessed.csv')
BM = report_df.year ==2019
report_df = pd.DataFrame(report_df[BM]).reset_index(drop=True)
report_df.drop(columns=['year'],inplace=True)
report_df.set_index('Name',inplace=True)
report_df

In [ ]:
Select_columns = ['Life_Ladder', 'Log_GDP_per_capita',
       'Social_support', 'Healthy_life_expectancy_at_birth',
       'Freedom_to_make_life_choices', 'Generosity',
       'Perceptions_of_corruption', 'Positive_affect', 'Negative_affect']
Xs = pd.DataFrame(report_df[Select_columns])

Xs = (Xs-Xs.min())/(Xs.max()-Xs.min())
Xs.describe()

# Data Clusteribility

## Hopkins Statistics 

In [ ]:
def hopkins(df,m):
    from sklearn.neighbors import NearestNeighbors
    from random import sample
    from pandas import DataFrame
    from numpy import random

    d = len(df.columns) # columns
    n = len(df) # rows
    
    df = (df - df.min())/(df.max()-df.min()) *2 -1
    df = df / df.std()
    

    knn = NearestNeighbors(n_neighbors=2).fit(df)

    rand_df = DataFrame(random.rand(m,d),index = range(0,m),columns =df.columns )
    rand_df = rand_df*2-1
    rand_df = rand_df * df.abs().max()

    ujd = []
    wjd = []

    for j in range(0, m):
        u_dist, _ = knn.kneighbors([rand_df.iloc[j]])
        ujd.append(u_dist[0][0])

        w_dist, _ = knn.kneighbors(df.sample(1))
        wjd.append(w_dist[0][1])

    return(sum(ujd) / (sum(ujd) + sum(wjd)))

In [ ]:
m = 10
hopkins(Xs,m)   

In [ ]:
for i in range(0,20):
    print(hopkins(Xs,m))

# K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4).fit(Xs)

# Cluster membership
memb = pd.Series(kmeans.labels_, index=Xs.index)
for key, item in memb.groupby(memb):
    print(key, ': ', ', '.join(item.index))

# Clustering Comparison

In [ ]:
def countpairs(Clustering1,Clustering2):
    from pandas import Series
    
    output = Series(0,index=['N00','N01','N10','N11'])

    for i in range(0,10):
        for j in range(0,i):
            if(i!=j):

                c1_same = False
                c2_same = False
                c1_Not_same = False
                c2_Not_same = False

                if(Clustering1[i]==Clustering1[j]):
                    c1_same=True
                else:
                    c1_Not_same=True
                if(Clustering2[i]==Clustering2[j]):
                    c2_same=True
                else:
                    c2_Not_same = True

                if(c1_same & c2_same):
                    output.N11 = output.N11 +1 
                if(c1_Not_same & c2_Not_same):
                    output.N00 = output.N00 +1
                if(c1_same & c2_Not_same):
                    output.N10 = output.N10 +1
                if(c1_Not_same & c2_same):
                    output.N01 = output.N01 +1

    return(output)

In [ ]:
Clustering1 = np.random.randint(1,4,len(Xs))
Clustering2 = np.random.randint(1,20,len(Xs))

countpairs(Clustering1,Clustering2)

## Fowlkes–Mallows

In [ ]:
def fowlkes_mallows(Clustering1,Clustering2):
    
    from numpy import sqrt
    from pandas import Series
    
    p = countpairs(Clustering1,Clustering2)
    
    return(p.N11/((p.N11+p.N01)+(p.N11+p.N10)))

In [ ]:
Clustering1 = np.random.randint(1,4,len(Xs))
Clustering2 = np.random.randint(1,4,len(Xs))

fowlkes_mallows(Clustering1,Clustering2)

In [ ]:
for i in range(0,20):
    Clustering1 = np.random.randint(1,4,22)
    Clustering2 = np.random.randint(1,4,22)
    
    print('fowlkes_mallows: {}'.format(fowlkes_mallows(Clustering1,Clustering2)))
    print('----------------')

In [ ]:
Clustering1 = np.random.randint(1,4,len(Xs))

print('fowlkes_mallows: {}'.format(fowlkes_mallows(Clustering1,Clustering1)))

# Measure K-Means consistency

In [ ]:
kmeans = KMeans(n_clusters=4)
Clustering1 = kmeans.fit(Xs).labels_
Clustering2 = kmeans.fit(Xs).labels_

print('fowlkes_mallows: {}'.format(fowlkes_mallows(Clustering1,Clustering2)))

In [ ]:
for i in range(0,20):
    Clustering1 = kmeans.fit(Xs).labels_
    Clustering2 = kmeans.fit(Xs).labels_
    
    print('fowlkes_mallows: {}'.format(fowlkes_mallows(Clustering1,Clustering2)))
    print('----------------')

# Find the number of clusters using SSE

In [ ]:
repetitions = ['R{}'.format(i) for i in range(1,10)]

SSE_results = pd.DataFrame(0.0, index = range(2,15), 
                       columns= repetitions)


for n_cluster in SSE_results.index:
    for col in SSE_results.columns:
        algort = KMeans(n_clusters=n_cluster).fit(Xs)
        SSE_results.at[n_cluster,col] = algort.inertia_ 
        # Inertia: Sum of distances of samples to their closest cluster center

SSE_results['Mean'] = SSE_results[repetitions].mean(axis=1)
SSE_results['Var'] = SSE_results[repetitions].var(axis=1)
SSE_results.sort_values('Mean')


In [ ]:
(SSE_results.Mean).plot()
plt.show()

# Find the number of clusters using Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score
algort = KMeans(n_clusters=3).fit(Xs)
silhouette_score(Xs,algort.labels_)

In [ ]:
repetitions = ['R{}'.format(i) for i in range(1,10)]

SIL_results = pd.DataFrame( index = range(2,25), 
                       columns= repetitions)


for n_cluster in SIL_results.index:
    for col in SIL_results.columns:
        algort = KMeans(n_clusters=n_cluster).fit(Xs)
        SIL_results.at[n_cluster,col] = silhouette_score(Xs,algort.labels_)
        
SIL_results['Mean'] = SIL_results[repetitions].mean(axis=1)
SIL_results['Var'] = SIL_results[repetitions].var(axis=1)
SIL_results.sort_values('Mean',ascending=False)

In [ ]:
SIL_results.Mean.plot()
plt.show()

In [ ]:
for i in range(0,10):
    Clustering1 = KMeans(n_clusters=3).fit(Xs).labels_
    Clustering2 = KMeans(n_clusters=3).fit(Xs).labels_
    
    print('fowlkes_mallows: {}'.format(fowlkes_mallows(Clustering1,Clustering2)))
    print('----------------')

Let's say we decide, k=3 is the best for this data. How do we know the output of which run should we us?

We run the algorithms for m times, and the run that leads to the highest similarity of patterns to the rest of the runs will be selected. 

In [ ]:
m=10

Clusterings =[]

for i in range(0,m):
    algort = KMeans(n_clusters=3, random_state=i).fit(Xs)
    Clusterings.append(algort.labels_)

Sim_Matrix = pd.DataFrame(0.0,index= ['Clustering{}'.format(i) for i in range(1,m+1)], 
                          columns=['Clustering{}'.format(i) for i in range(1,m+1)])
for i, inx in enumerate(Sim_Matrix.index):
    for j, col in enumerate(Sim_Matrix.columns):
        Sim_Matrix.at[inx,col] = fowlkes_mallows(Clusterings[i],Clusterings[j])

Sim_Matrix

In [ ]:
plt.figure(figsize=(10,6))

sns.heatmap(Sim_Matrix, linewidths=.5, annot=True, 
                    cmap='Greens')
plt.show()

In [ ]:
memb = pd.Series(Clusterings[2], index=Xs.index)
for key, item in memb.groupby(memb):
    print(key, ': ', ', '.join(item.index))

# centroid Analysis

In [ ]:
clusters = ['Cluster {}'.format(i) for i in range(3)]
Centroids_orig = pd.DataFrame(0.0, index = clusters,
                        columns = report_df.columns)

Centroids_std = pd.DataFrame(0.0, index =  clusters,
                        columns = Xs.columns)
for i in range(3):
    BM = memb==i
    Centroids_orig.iloc[i] = report_df[BM].median(axis=0)
    Centroids_std.iloc[i] = Xs[BM].mean(axis=0)
    
Centroids_orig

In [ ]:
sns.heatmap(Centroids_std, linewidths=.5, annot=True, 
                    cmap='Purples')
plt.show()

In [ ]:
replace_dic = {0:'Happy and crime-ridden',
               1:'Unhappy and crime-ridden',
               2:'Very happy'}
report_df['Cluster'] = memb.replace(replace_dic)
report_df

# More analysis 
Investigate the relationship between the attribute cluster and the attributes we didn't used for clustering

In [ ]:
contingency_tbl = pd.crosstab(report_df.Cluster, report_df.Continent)
probablity_tbl = contingency_tbl/ contingency_tbl.sum()
sns.heatmap(probablity_tbl, annot=True, center=0.5 ,cmap="Greys")
plt.show()

In [ ]:
contingency_tbl = pd.crosstab(report_df.Continent, report_df.Cluster)
probablity_tbl = contingency_tbl/ contingency_tbl.sum()
sns.heatmap(probablity_tbl, annot=True, center=0.5 ,cmap="Greys")
plt.show()

In [ ]:
pop_discretized = pd.cut(report_df.population, bins = 10)
contingency_tbl = pd.crosstab(report_df.Cluster,pop_discretized)
probablity_tbl = contingency_tbl/ contingency_tbl.sum()
sns.heatmap(probablity_tbl, annot=True, center=0.5 ,cmap="Greys")
plt.show()

In [ ]:
plt.scatter(report_df.population,
           memb.apply(lambda v: v+np.random.random()/1.8),
           marker='.')
plt.yticks([0,1,2],['Happy and crime-ridden',
                     'Unhappy and crime-ridden',
                     'Very happy'])
plt.show()